# Ejercicio 4: Modelo Probabilístico

## Objetivo de la práctica
- Comprender los componentes del modelo vectorial mediante cálculos manuales y observación directa.
- Aplicar el modelo de espacio vectorial con TF-IDF para recuperar documentos relevantes.
- Comparar la recuperación con BM25 frente a TF-IDF.
- Analizar visualmente las diferencias entre los modelos.
- Evaluar si los rankings generados son consistentes con lo que considerarías documentos relevantes.

## Parte 0: Carga del Corpus

In [1]:
from sklearn.datasets import fetch_20newsgroups

newsgroups = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))
newsgroupsdocs = newsgroups.data

## Parte 1: Cálculo de TF, DF, IDF y TF-IDF

### Actividad 
1. Utiliza el corpus cargado.
2. Construye la matriz de términos (TF), y calcula la frecuencia de documentos (DF)
3. Calcula TF-IDF utilizando sklearn.
4. Visualiza los valores en un DataFrame para analizar las diferencias entre los términos.

In [4]:
import pandas as pd
df = pd.DataFrame(newsgroupsdocs)
df

,0
0,\n\nI am sure some bashers of Pens fans are pr...
1,My brother is in the market for a high-perform...
2,\n\n\n\n\tFinally you said what you dream abou...
3,\nThink!\n\nIt's the SCSI card doing the DMA t...
4,1) I have an old Jasmine drive which I cann...
...,...
18841,DN> From: nyeda@cnsvax.uwec.edu (David Nye)\nD...
18842,\nNot in isolated ground recepticles (usually ...
18843,I just installed a DX2-66 CPU in a clone mothe...
18844,\nWouldn't this require a hyper-sphere. In 3-...


In [5]:
from preprocessing import preprocess_text

df['prep'] = df[0].apply(preprocess_text)
df

,0,prep
0,\n\nI am sure some bashers of Pens fans are pr...,i am sur som bashers of pens fans are pretty c...
1,My brother is in the market for a high-perform...,my broth is in the market for high performanc ...
2,\n\n\n\n\tFinally you said what you dream abou...,finally you said what you dream about mediterr...
3,\nThink!\n\nIt's the SCSI card doing the DMA t...,think it s the scsi card doing the dma transfe...
4,1) I have an old Jasmine drive which I cann...,1 i hav an old jasmin driv which i cannot use ...
...,...,...
18841,DN> From: nyeda@cnsvax.uwec.edu (David Nye)\nD...,dn from nyed cnsvax uwec edu dav nye dn neurol...
18842,\nNot in isolated ground recepticles (usually ...,not in isolat ground recepticl usually an unus...
18843,I just installed a DX2-66 CPU in a clone mothe...,i just install dx2 66 cpu in clon motherboard ...
18844,\nWouldn't this require a hyper-sphere. In 3-...,wouldn t this requir hyper spher in 3 spac 4 p...


## Parte 2: Ranking de documentos usando TF-IDF

### Actividad 

1. Dada una consulta, construye el vector de consulta
2. Calcula la similitud coseno entre la consulta y cada documento usando los vectores TF-IDF
3. Genera un ranking de los documentos ordenados por relevancia.
4. Muestra los resultados en una tabla.

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

X = vectorizer.fit_transform(df['prep'])

In [29]:
query = 'government'
q_proc = preprocess_text(query)
q_vec = vectorizer.transform([query])

In [30]:
from sklearn.metrics.pairwise import cosine_similarity

df['q_sim'] = cosine_similarity(X, q_vec)
df.sort_values(by='q_sim', ascending=False)

,0,prep,q_sim
5905,\tYou have underlined here the battle that mus...,you hav underlin her the battl that must be fo...,0.377337
15813,>government maintaining a secret of some kind.,government maintaining secret of som kind,0.376554
8841,"\nSaw it in t.p.m., thanks.\n\n\n\n\nYes, and ...",saw it in t p m thanks yes and the individual ...,0.373465
13956,\nDoesn't this just mean that the government m...,doesn t this just mean that the government mig...,0.344338
5225,\n\nI suspect the government feels it is enoug...,i suspect the government feels it is enough to...,0.344235
...,...,...,...
6446,"Another company, Congruent Corporation of New ...",anoth company congruent corporation of new yor...,0.000000
6447,The people who post to this particular newsgro...,the peopl who post to this particul newsgroup ...,0.000000
6448,"\nMy reason for saying that was 'cause _I_, if...",my reason for saying that was caus i if i wer ...,0.000000
6449,How come noone mentions Eric Hoffer when talki...,how com noon mentions eric hoff when talking a...,0.000000


## Parte 3: Ranking con BM25

### Actividad 

1. Implementa un sistema de recuperación usando el modelo BM25.
2. Usa la misma consulta del ejercicio anterior.
3. Calcula el score BM25 para cada documento y genera un ranking.
4. Compara manualmente con el ranking de TF-IDF.

Algoritmo BM25:

1. Tokenización
2. Calcular longitudes de documentos y promedio de longitud
3. Calcular TF (frecuencia de término por documento)
4. Calcular DF (document frequency) e IDF de BM25
5. Definir los parámetros de BM25 (k1, b)
6. Implementar la función de score BM25 para un documento
7. Función para calcular la probabilidad para todos los documentos de la colección a partir de una query
8. Recuperar los documentos mejor puntuados (ranking)

## Parte 4: Comparación visual entre TF-IDF y BM25

### Actividad 

1. Utiliza un gráfico de barras para visualizar los scores obtenidos por cada documento según TF-IDF y BM25.
2. Compara los rankings visualmente.
3. Identifica: ¿Qué documentos obtienen scores más altos en un modelo que en otro?
4. Sugiere: ¿A qué se podría deber esta diferencia?

## Parte 5: Evaluación con consulta relevante

### Actividad 

1. Elige una consulta y define qué documentos del corpus deberían considerarse relevantes.
2. Evalúa Precision@3 o MAP para los rankings generados con TF-IDF y BM25.
3. Responde: ¿Cuál modelo da mejores resultados respecto a tu criterio de relevancia?